# The most significant adjectives for men / women

__What's the difference in describing men and women?__

Which can be translated to more technical terms: 

__What are the most significant adjectives used with the word 'man' as opposed to the word 'woman'?__


The general approach would be:
- use the standard NLTK corpora representing free text (eg gutenberg, brown, webtext)
- count the occurences of every word preceding 'man' or 'woman' (let's call them adjectives)
- sort by the 'most significant' adjective, that is one that is most 'uncommonly common' for a given category - man or woman
- show them

## Technologies
- **NLTK** for NLP
- **pandas** for 'most-significant' calculations

In [110]:
import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
import pandas as pd

def most_common_adj(corpus, word, count=10):
    # bigrams ending with word
    bigrams = nltk.bigrams(corpus.words())
    fd = nltk.FreqDist([w for w in [x.lower() for x,y in list(bigrams) if y==word] if len(w)>2 and w not in stopwords])
    return fd.most_common(count)
    
words = ["man", "woman"]
corpuses = [nltk.corpus.gutenberg, nltk.corpus.brown, nltk.corpus.webtext]
most_common = [[most_common_adj(corpus, word, count=1000) for word in words] for corpus in corpuses]

print [[word_counts[:10] for word_counts in corpus] for corpus in most_common]

[[[(u'every', 327), (u'young', 266), (u'old', 207), (u'one', 72), (u'another', 42), (u'little', 38), (u'wise', 34), (u'certain', 33), (u'good', 31), (u'poor', 30)], [(u'old', 61), (u'young', 57), (u'every', 10), (u'strange', 9), (u'poor', 9), (u'certain', 8), (u'another', 7), (u'good', 6), (u'lovely', 6), (u'charming', 6)]], [[(u'old', 53), (u'young', 47), (u'one', 25), (u'big', 17), (u'fat', 11), (u'every', 10), (u'good', 9), (u'little', 9), (u'white', 8), (u'common', 7)], [(u'old', 13), (u'young', 11), (u'another', 5), (u'every', 3), (u'strange', 3), (u'one', 2), (u'pink', 2), (u'lonely', 2), (u'strong', 2), (u'american', 2)]], [[(u'old', 86), (u'black', 19), (u'white', 16), (u'young', 12), (u'hey', 10), (u'blind', 9), (u'yeah', 8), (u'tourist', 7), (u'good', 6), (u'vendor', 5)], [(u'old', 51), (u'black', 26), (u'young', 23), (u'tourist', 13), (u'southern', 12), (u'aged', 10), (u'white', 9), (u'drunk', 7), (u'italian', 6), (u'ghetto', 5)]]]


In [112]:

def most_significant(groups, limit=10):
    """ Sorts adjectives in each group with most significant for the group first
    
        most significant means occurring in the group more often than in the whole corpus
        significance ranges from [0 to 1], 1 meaning that the adjective doesn't exist anywhere else but in this group
        NOTE: if two words have the same significance then the one occurring more often is more significant
    """
    dfs = [pd.DataFrame(group, columns=['word', 'counts']) for group in groups]
    dfs = [df.set_index('word') for df in dfs]
    
    #     adjustment is required to remove bias caused by some groups being more populous
    biggest_count = max([sum(df.counts) for df in dfs])
    dfs = [df*biggest_count/sum(df.counts) for df in dfs]
    
    total = reduce(lambda x,y: x.add(y, fill_value=0), dfs)
    for df in dfs:
        df['ratio'] = df/total
        
    sigs = [df.dropna().to_records().tolist() for df in dfs]
    sorted_sigs = [ sorted(sig, key=lambda x: (x[2],x[1]), reverse=True) for sig in sigs ]
    return [ sig[:limit] for sig in sorted_sigs ]
    

for corpus, corpus_most_common in zip(corpuses, most_common):
    print "\n-------------------------------------\n{}\n-------------------------------------\n".format(corpus.readme().split('\n')[0])
    for word, word_most_significant in zip(words, most_significant(corpus_most_common, limit=100)):
#         print "\n{}: -------------\n{}".format(word, word_most_significant)
            print "\n{}: -------------\n{}".format(word, ",".join(map(lambda x: x[0], word_most_significant)))



-------------------------------------
Project Gutenberg Selections
-------------------------------------


man: -------------
mighty,righteous,honest,blind,lazy,first,white,euery,strong,tall,prudent,honourable,dead,valiant,never,grey,last,violent,upright,slothful,small,brave,third,drunken,industrious,neither,thou,without,unto,big,made,average,faithful,whatsoever,large,new,hairy,clever,known,perfect,haired,free,created,like,lean,wayfaring,mean,sick,bloody,armed,crazy,lame,short,furious,inward,unhappy,happiest,dumb,best,faced,yet,fellow,impatient,teneriffe,deceitful,minded,pious,spiritual,tired,public,holy,unjust,mad,chief,chosen,travelling,hard,absentminded,wronged,silent,gentlemanlike,austere,aged,impotent,though,astonished,nervous,black,lucky,crooked,monstrous,fearless,devout,slain,goodly,plain,engaged,bearded,bald,infatuated

woman: -------------
lovely,widow,virtuous,midianitish,israelitish,amiable,whorish,valuable,brawling,canaanitish,delightful,pleasant,ethiopian,menstruous,trava